<!-- @format -->

Zadanie 4. (6p) W tym zadaniu będziemy generować zdania w języku naturalnym spełniające
dodatkową właściwość: wszystkie słowa powinny zaczynać się od tej samej litery. Na SKOS
będzie dostępna lista prefiksów, takich jak: „Prawdziwy piekarz przyprawia pieczywo pieprzem”,
Twoje generacje powinny zawsze zaczynać się od wylosowanego prefiksu i kończyć kropką (lub innym
znakiem interpunkcjnym kończącym zdanie).
Twój program powinien:

1. Stosować zarówno top-k, jak i top-p.
2. Modyfikować rozkład prawdopodobieństwa tokenów
3. Generować „porządne” teksty, to znaczy z wyrazami składającymi się z liter, ze spacjami jako
   separatorami, z interpunkcją poprawnie formatowaną (na przykład przecinek powinien być
   doklejony do poprzedzającego go słowa)
4. Unikać powtórzeń
5. Generować kilka wariantów i wybierać z nich najlepszy, wg zdefiniowanego przez Ciebie kryterium
   Wygenerowany tekst


In [131]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset


device = "cuda" if torch.cuda.is_available else "cpu"
model_name = "flax-community/papuGaPT2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
device

'cuda'

In [ ]:
from random import randint, seed


def load_sample():
    with open("prefiksy.txt", "r", encoding="utf-8") as prefixes:

        lines = prefixes.readlines()
        return list(set([line.rstrip() for line in lines]))


def random_prefix(sample, set_seed=False):
    if set_seed:
        seed(0)
    size = len(sample)
    row_num = randint(0, size - 1)
    # return (lines[row_num].rstrip(), row_num+1)
    return sample[row_num]


sample = load_sample()
r_prefix = random_prefix(sample)
r_prefix

'Po prelekcji przewidziana'

In [108]:
# Save the sample dataset to a file
with open("training_data.txt", "w", encoding="utf-8") as f:
    for line in sample:
        f.write(line + "\n")

<!-- @format -->

### Plik


In [ ]:
# Prepare dataset
def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer, file_path=file_path, block_size=block_size
    )
    return dataset


# Data collator
def data_collator(tokenizer):
    return DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


# Load dataset
train_dataset = load_dataset("training_data.txt", tokenizer)

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator(tokenizer),
    train_dataset=train_dataset,
)

# Fine-tune model
trainer.train()

<!-- @format -->

### Lista


In [ ]:
tokenizer.pad_token = tokenizer.eos_token


# Prepare dataset from list
def create_dataset_from_list(texts):
    data = {"text": texts}
    dataset = Dataset.from_dict(data)
    return dataset.map(
        lambda examples: tokenizer(
            examples["text"], truncation=True, padding="max_length", max_length=128
        ),
        batched=True,
    )


# Load dataset
train_dataset = create_dataset_from_list(sample)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Fine-tune model
trainer.train()

Map:   0%|          | 0/44891 [00:00<?, ? examples/s]

  0%|          | 0/33669 [00:00<?, ?it/s]

{'loss': 5.4637, 'grad_norm': 44.26801300048828, 'learning_rate': 4.925747720455019e-05, 'epoch': 0.04}


KeyboardInterrupt: 

In [128]:
def generate_text(prefix, top_k, top_p, temp=0.8, max_length=20):
    inputs_ids = tokenizer.encode(prefix, return_tensors="pt").to(device)

    output = model.generate(
        inputs_ids,
        max_length=max_length,
        do_sample=True,
        top_k=top_k,
        top_p=top_p,
        temperature=temp,
        num_return_sequences=5,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2,
    )

    return [tokenizer.decode(output[i], skip_special_tokens=True) for i in range(5)]


def filter_text(generated_texts):

    filtered_texts = [
        text
        for text in generated_texts
        if all(word.startswith(generated_texts[0]) for word in text.split())
    ]

    if not filtered_texts:
        return "No valid text generated."

    best_text = max(filtered_texts, key=lambda text: len(set(text.split())))
    return best_text


generate_text(r_prefix, 20, 80)
# filtered_text = filter_text(texts)
# generate_text

# %%

['Po prelekcji przewidziana\nPrzedstawienie przygotowane przez,Pacjent powinien: posiadać pełną zdolność do\n',
 'Po prelekcji przewidziana\nZachowanie przepisów prawa,wspomniany przepis przejściowy powoduje:',
 'Po prelekcji przewidziana\nDołączył do drużyny, którąPoczątkowo przedsiębiorstwo prowadziło początkowo pod nazwą Pa',
 'Po prelekcji przewidziana\nPan poseł Piechowiak,Prokurator GeneralnyPragnąłbym przedstawić',
 'Po prelekcji przewidziana\nPrzedłożony projekt, popartaPracąPaństwaPrawnicząProponuję']

In [113]:
def generate_diverse_text(
    prefix, model, tokenizer, top_k=50, top_p=0.9, max_length=100, num_variants=5
):
    input_ids = tokenizer.encode(prefix, return_tensors="pt").to(device)
    generated_texts = []

    for _ in range(num_variants):
        output = model.generate(
            input_ids,
            max_length=max_length,
            do_sample=True,
            top_k=top_k,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
            num_return_sequences=1,
        )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_texts.append(generated_text)

    # Post-process to ensure all words start with the same letter and proper formatting
    filtered_texts = [
        text
        for text in generated_texts
        if all(word.startswith(prefix[0]) for word in text.split())
    ]

    if not filtered_texts:
        return "No valid text generated."

    best_text = max(
        filtered_texts, key=lambda text: len(set(text.split()))
    )  # Example criterion: maximize unique words

    return best_text


# Example usage
prefix = "Prawdziwy piekarz przyprawia pieczywo pieprzem"
generated_text = generate_diverse_text(prefix, model, tokenizer)
print(generated_text)

No valid text generated.


<!-- @format -->

## Giga skibidi plan działania

1. Rodzielam dane na train i validation
2. Robie fine tuning
3. Dla wylosowanego prefiksu sprawdzam czy nie ma go w train
4. Wypluwam rozwiązanie
